In [8]:
# chapter 3-2 webページを簡単に取得する

In [1]:
import requests

In [2]:
r = requests.get('https://gihyo.jp/dp')
# webページを取得

In [3]:
type(r)
# get関数の戻り値はレスポンス型

requests.models.Response

In [5]:
r.status_code
# HTTPステータスコードを取得

200

In [6]:
r.headers['content-type']
# HTTPヘッダーの辞書を取得

'text/html; charset=UTF-8'

In [7]:
r.encoding
# HTTPヘッダーから得られたエンコーディングを取得

'UTF-8'

In [ ]:
r.text
# str型にデコードしたレスポンスボディを取得

In [ ]:
r.content
# bytes型のレスポンスボディを取得

In [14]:
r = requests.get('http://weather.livedoor.com/forecast/webservice/json/v1?city=130010')
# 東京の天気をjson形式で取得

In [ ]:
r.json()

In [16]:
r = requests.post('http://httpbin.org/post', data={'key1': 'value1'})
# POSTメソッドで送信

In [19]:
r = requests.get('http://httpbin.org/get',
                headers={'user-agent': 'my-crawler/1.0 (+foo@example.com)'})
# キーワード引数headersにdictで指定してリクエストにHTTPヘッダーを追加

In [22]:
r = requests.get('https://api.github.com/user',
                auth=('discocactus', '<password>'))
# Basic認証のユーザー名とパスワードの組をキーワード引数authで指定

In [24]:
r = requests.get('http://httpbin.org/get', params={'key1': 'value1'})
# URLのパラメーターは引数paramsで指定することも可能

In [34]:
# 複数のページを連続してクロールする場合は、Sessionオブジェクトを使うのが効果的

In [26]:
s = requests.Session()

In [29]:
s.headers.update({'user-agent': 'my-crawler/1.0 (+foo@example.com)'})
# HTTPヘッダーを複数のリクエストで使い回す

In [30]:
r = s.get('https://gihyo.jp/')
# Sessionオブジェクトでもrequestsの様にget(), post()などのメソッドが使える

In [31]:
r = s.get('https://gihyo.jp/dp')

In [33]:
# chapter 3-3 HTMLのスクレイピング

In [59]:
# lxmlによるスクレイピング

In [35]:
import lxml.html

In [36]:
tree = lxml.html.parse('index.html')
# parse()関数でファイルパスを指定してパース
# URLを指定することも可能だが細かい指定ができない

In [37]:
from urllib.request import urlopen

In [38]:
tree = lxml.html.parse(urlopen('http://example.com/'))
# ファイルオブジェクトを指定してパースすることも可能

In [39]:
type(tree)

lxml.etree._ElementTree

In [40]:
html = tree.getroot()

In [41]:
type(html)

lxml.html.HtmlElement

In [42]:
# fromstring()関数で文字列(strまたはbytes)をパースできる
# ただし、encodingが指定されたXML宣言を含むstrをパースすると、ValueErrorが発生するので注意

In [72]:
html = lxml.html.fromstring('''
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id="main">今日のくだもの</h1>
    <ul>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ul>
    </body>
    </html>''')

In [44]:
type(html)

lxml.html.HtmlElement

In [73]:
html.xpath('//li')
# XPathにマッチする要素のリストを取得

[<Element li at 0x10e7d9c28>,
 <Element li at 0x10e7d9c78>,
 <Element li at 0x10e7d9cc8>]

In [74]:
html.cssselect('li')
# CSSセレクターにマッチする要素のリストを取得

[<Element li at 0x10e7d9c28>,
 <Element li at 0x10e7d9c78>,
 <Element li at 0x10e7d9cc8>]

In [49]:
h1 = html.xpath('//h1')[0]

In [50]:
h1.tag
# タグの名前

'h1'

In [51]:
h1.text
# 要素のテキスト

'今日のくだもの'

In [52]:
h1.get('id')
# 属性の値

'main'

In [53]:
h1.attrib
# 全属性を表すdict-likeなオブジェクト

{'id': 'main'}

In [54]:
h1.getparent()
# 親要素

<Element body at 0x10e6bc3b8>

In [57]:
%%writefile scrape_by_lxml.py

import lxml.html

# HTMLファイルを読み込みgetroot()メソッドでHtmlElementオブジェクトを得る
tree = lxml.html.parse('index.html')
html = tree.getroot()

# cssselect()メソッドでa要素のリストを取得して、個々のa要素に対して処理を行う
for a in html.cssselect('a'):
    # href属性とリンクのテキストを取得して表示する
    print(a.get('href'), a.text)

Overwriting scrape_by_lxml.py


In [ ]:
!python scrape_by_lxml.py

In [60]:
# Beautiful Soupによるスクレイピング

In [61]:
from bs4 import BeautifulSoup

In [62]:
with open('index.html') as f:
    soup = BeautifulSoup(f, 'html.parser')
# 第1引数にファイルオブジェクトを指定してBeautifulSoupオブジェクトを生成
# BeautifulSoupにはファイル名やURLを指定することはできない
# 第2引数にパーサーを指定する

In [70]:
# BeautifulSoupのコンストラクターにはHTMLの文字列を渡すことも可能
soup = BeautifulSoup('''
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id="main">今日のくだもの</h1>
    <ul>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ul>
    </body>
    </html>''', 'html.parser')

In [64]:
soup.h1

<h1 id="main">今日のくだもの</h1>

In [65]:
type(soup.h1)

bs4.element.Tag

In [66]:
soup.h1.name

'h1'

In [67]:
soup.h1.string

'今日のくだもの'

In [68]:
type(soup.h1.string)

bs4.element.NavigableString

In [71]:
soup.ul.text

'\nりんご\nみかん\nぶどう\n'

In [75]:
type(soup.h1.text)

str

In [76]:
soup.h1['id']

'main'

In [77]:
soup.h1.get('id')

'main'

In [78]:
soup.h1.attrs

{'id': 'main'}

In [79]:
soup.h1.parent

<body>
<h1 id="main">今日のくだもの</h1>
<ul>
<li>りんご</li>
<li class="featured">みかん</li>
<li>ぶどう</li>
</ul>
</body>

In [80]:
soup.li
# 複数の要素がある場合は先頭の要素が取得される

<li>りんご</li>

In [81]:
soup.find('li')

<li>りんご</li>

In [83]:
soup.find_all('li')
# find_all()メソッドで指定した名前の要素のリストを取得

[<li>りんご</li>, <li class="featured">みかん</li>, <li>ぶどう</li>]

In [84]:
soup.find_all('li', class_='featured')
# キーワード引数でclassなどの属性を指定できる
# classは予約語なのでclass_を使うことに注意

[<li class="featured">みかん</li>]

In [85]:
soup.find_all(id='main')

[<h1 id="main">今日のくだもの</h1>]

In [87]:
soup.find_all('li', class_='featured')

[<li class="featured">みかん</li>]

In [88]:
soup.find_all(id='main')
# タグ名を省略して属性のみで探すことも可能

[<h1 id="main">今日のくだもの</h1>]

In [86]:
soup.select('li')
# select()メソッドでCSSセレクターにマッチする要素を取得

[<li>りんご</li>, <li class="featured">みかん</li>, <li>ぶどう</li>]

In [90]:
soup.select('li.featured')

[<li class="featured">みかん</li>]

In [91]:
soup.select('#main')

[<h1 id="main">今日のくだもの</h1>]

In [92]:
%%writefile scrape_by_bs4.py

from bs4 import BeautifulSoup

with open('index.html') as f:
    soup = BeautifulSoup(f, 'html.parser')

for a in soup.find_all('a'):
    print(a.get('href'), a.text)

Writing scrape_by_bs4.py


In [ ]:
!python scrape_by_bs4.py

In [95]:
# pyqueryによるスクレイピング

In [96]:
from pyquery import PyQuery as pq

In [97]:
d = pq(filename='index.html')

In [98]:
d = pq(url='http://example.com/')

In [99]:
d = pq('''
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id="main">今日のくだもの</h1>
    <ul>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ul>
    </body>
    </html>''')

In [100]:
d('h1')

[<h1#main>]

In [101]:
type(d('h1'))

pyquery.pyquery.PyQuery

In [102]:
d('h1')[0]

<Element h1 at 0x10e654988>

In [103]:
d('h1').text()

'今日のくだもの'

In [104]:
d('h1').attr('id')

'main'

In [105]:
d('h1').attr.id

'main'

In [106]:
d('h1').attr['id']

'main'

In [107]:
d('h1').parent()

[<body>]

In [108]:
d('li')

[<li>, <li.featured>, <li>]

In [109]:
d('li.featured')

[<li.featured>]

In [110]:
d('#main')

[<h1#main>]

In [111]:
d('body').find('li')

[<li>, <li.featured>, <li>]

In [112]:
d('li').filter('.featured')

[<li.featured>]

In [116]:
d('li').eq(1)

[<li.featured>]

In [117]:
# RSSのスクレイピング

In [1]:
import feedparser

In [7]:
d = feedparser.parse('http://b.hatena.ne.jp/hotentry/it.rss')
# parse()関数にURLを指定してパースできる

In [4]:
# d = feedparser.parse('it.rss')
# parse()関数にはファイルパス、ファイルオブジェクト、XMLの文字列も指定できる

In [8]:
type(d)

feedparser.FeedParserDict

In [9]:
d.version

'rss10'

In [10]:
d.feed.title

'はてなブックマーク - 人気エントリー - テクノロジー'

In [11]:
d['feed']['title']
# dictの形式でもアクセスできる

'はてなブックマーク - 人気エントリー - テクノロジー'

In [12]:
d.feed.link

'http://b.hatena.ne.jp/hotentry/it'

In [13]:
d.feed.description
# フィードの説明を取得する

'最近の人気エントリー - テクノロジー'

In [14]:
len(d.entries)

30

In [15]:
d.entries[0].title

'一部報道につきまして。株式会社CAMPFIRE | CAMPFIRE MAGAZINE'

In [16]:
d.entries[0].link

'https://mag.camp-fire.jp/21319/'

In [17]:
d.entries[0].description

'CAMPFIREに掲載しておりますプロジェクトにつきまして、一部情報サイト等で報道がされておりますが、株式会社CAMPFIREからの見解を発表させていただきます。 ・CAMPFIREはプラットフォームであり目標金額をこちらから無理に設定させ...CAMPFIREに掲載しておりますプロジェクトにつきまして、一部情報サイト等で報道がされておりますが、株式会社CAMPFIREからの見解を発表させていただ...'

In [19]:
d.entries[0].updated

'2017-08-30T15:15:08+09:00'

In [20]:
d.entries[0].updated_parsed
# 要素の更新日時をパースしてtime.struct_timeを取得する

time.struct_time(tm_year=2017, tm_mon=8, tm_mday=30, tm_hour=6, tm_min=15, tm_sec=8, tm_wday=2, tm_yday=242, tm_isdst=0)

In [21]:
%%writefile scrape_by_feedparser.py

import feedparser

d = feedparser.parse('http://b.hatena.ne.jp/hotentry/it.rss')

for entry in d.entries:
    print(entry.link, entry.title)

Writing scrape_by_feedparser.py


In [ ]:
!python scrape_by_feedparser.py

In [23]:
# chapter 3-5 データベースに保存する

In [24]:
# MySQLへのデータの保存

In [25]:
# データベースとユーザーの作成 > ターミナル操作はEvernoteに

In [15]:
%%writefile save_mysql.py

import MySQLdb

conn = MySQLdb.connect(db='scraping', user='scraper', passwd='password', charset='utf8mb4')

c = conn.cursor()
c.execute('DROP TABLE IF EXISTS cities')
c.execute('''
    CREATE TABLE cities (
        rank integer,
        city text,
        population integer
    )
''')

c.execute('INSERT INTO cities VALUES (%s, %s, %s)', (1, '上海', 24150000))

c.execute('INSERT INTO cities VALUES (%(rank)s, %(city)s, %(population)s)',
         {'rank': 2, 'city': 'カラチ', 'population': 23500000})

c.executemany('INSERT INTO cities VALUES (%(rank)s, %(city)s, %(population)s)', [
    {'rank': 3, 'city': '北京', 'population': 21516000},
    {'rank': 4, 'city': '天津', 'population': 14722100},
    {'rank': 5, 'city': 'イスタンブル', 'population': 14160467},
])

conn.commit()

c.execute('SELECT * FROM cities')
for row in c.fetchall():
    print(row)
    
conn.close()

Overwriting save_mysql.py


In [16]:
!python save_mysql.py

(1, '上海', 24150000)
(2, 'カラチ', 23500000)
(3, '北京', 21516000)
(4, '天津', 14722100)
(5, 'イスタンブル', 14160467)


In [17]:
# MongoDBへのデータの保存

In [1]:
# 本ではデフォルトのデータベースディレクトリは /data/db 、起動は mongod コマンドのみだが、それではなぜかうまく動かない
# ターミナルで mongod --config /usr/local/etc/mongod.conf で起動
# 終了は ctrl + c

In [4]:
%%writefile save_mongo.py

import lxml.html
from pymongo import MongoClient

# HTMLファイルを読み込み、getroot()メソッドでHtmlElementオブジェクトを得る
tree = lxml.html.parse('index.html')
html = tree.getroot()

client = MongoClient('localhost', 27017)
db = client.scraping # scrapingデータベースを取得（作成）する
collection = db.links # linksコレクションを取得（作成）する

# このスクリプトを何回実行しても同じ結果になるよう、コレクションのドキュメントをすべて削除する
collection.delete_many({})

# cssselect()メソッドでa要素のリストを取得して、個々のa要素に対して処理を行う
for a in html.cssselect('a'):
    # href属性とリンクのテキストを取得して保存する
    collection.insert_one({
        'url': a.get('href'),
        'title': a.text,
    })

# コレクションのすべてのドキュメントを_idの順にソートして取得する
for link in collection.find().sort('_id'):
    print(link['_id'], link['url'], link['title'])

Overwriting save_mongo.py


In [ ]:
!python save_mongo.py

In [16]:
# chapter 3-6 クローラーとURL

In [7]:
# 相対URLから絶対URLへの変換例

In [8]:
from urllib.parse import urljoin

In [9]:
base_url = 'http://example.com/books/top.html'

In [10]:
# // で始まる相対URL
urljoin(base_url, '//cdn.example.com/logo.png')

'http://cdn.example.com/logo.png'

In [11]:
# / で始まる相対URL
urljoin(base_url, '/articles/')

'http://example.com/articles/'

In [12]:
# ./ 形式の表記
urljoin(base_url, './')

'http://example.com/books/'

In [15]:
# chapter 3-7 Pythonによるクローラーの作成

In [20]:
%%writefile python_crawler_1.py

# 一覧ページからURLの一覧を抜き出す(1)

import requests
import lxml.html

response = requests.get('https://gihyo.jp/dp')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('a[itemprop="url"]'):
    url = a.get('href')
    print(url)

Overwriting python_crawler_1.py


In [23]:
!python python_crawler_1.py

/dp/ebook/2017/978-4-7741-9243-7
/dp/ebook/2017/978-4-7741-9244-4
/dp/ebook/2017/978-4-7741-9240-6
/dp/ebook/2017/978-4-7741-9204-8
/dp/ebook/2017/978-4-7741-9198-0
/dp/ebook/2017/978-4-7741-8851-5
/dp/ebook/2017/978-4-7741-9233-8
/dp/ebook/2017/978-4-7741-9241-3
/dp/ebook/2017/978-4-7741-9231-4
/dp/ebook/2017/978-4-7741-9232-1
/dp/ebook/2017/978-4-7741-9229-1
/dp/ebook/2017/978-4-7741-9230-7
/dp/ebook/2017/978-4-7741-9228-4
/dp/ebook/2017/978-4-7741-9227-7
/dp/ebook/2017/978-4-7741-9206-2
/dp/ebook/2017/978-4-7741-9205-5
/dp/ebook/2017/978-4-7741-8845-4
/dp/ebook/2017/978-4-7741-9201-7
/dp/ebook/2017/978-4-7741-9203-1
/dp/ebook/2017/978-4-7741-9196-9
/dp/ebook/2017/978-4-7741-9202-4
/dp/ebook/2017/978-4-7741-9200-0
/dp/ebook/2017/978-4-7741-9197-3
/dp/ebook/2017/978-4-7741-9199-7
/dp/ebook/2017/978-4-7741-9195-9
/dp/ebook/2017/978-4-7741-9192-8
/dp/ebook/2017/978-4-7741-9191-1
/dp/ebook/2017/978-4-7741-9163-8
/dp/ebook/2017/978-4-7741-9190-4
/dp/ebook/2017/978-4-7741-9188-1
/dp
/dp/my

In [21]:
%%writefile python_crawler_2.py

# 一覧ページからURLの一覧を抜き出す(2)
# 不要なリンクを除外し、相対URLを絶対URLに変換する

import requests
import lxml.html

response = requests.get('https://gihyo.jp/dp')
root = lxml.html.fromstring(response.content)
root.make_links_absolute(response.url) # すべてのリンクを絶対URLに変換する

# id="listBook"である要素の子孫のa要素のみを取得する
for a in root.cssselect('#listBook a[itemprop="url"]'):
    url = a.get('href')
    print(url)

Writing python_crawler_2.py


In [22]:
!python python_crawler_2.py

https://gihyo.jp/dp/ebook/2017/978-4-7741-9243-7
https://gihyo.jp/dp/ebook/2017/978-4-7741-9244-4
https://gihyo.jp/dp/ebook/2017/978-4-7741-9240-6
https://gihyo.jp/dp/ebook/2017/978-4-7741-9204-8
https://gihyo.jp/dp/ebook/2017/978-4-7741-9198-0
https://gihyo.jp/dp/ebook/2017/978-4-7741-8851-5
https://gihyo.jp/dp/ebook/2017/978-4-7741-9233-8
https://gihyo.jp/dp/ebook/2017/978-4-7741-9241-3
https://gihyo.jp/dp/ebook/2017/978-4-7741-9231-4
https://gihyo.jp/dp/ebook/2017/978-4-7741-9232-1
https://gihyo.jp/dp/ebook/2017/978-4-7741-9229-1
https://gihyo.jp/dp/ebook/2017/978-4-7741-9230-7
https://gihyo.jp/dp/ebook/2017/978-4-7741-9228-4
https://gihyo.jp/dp/ebook/2017/978-4-7741-9227-7
https://gihyo.jp/dp/ebook/2017/978-4-7741-9206-2
https://gihyo.jp/dp/ebook/2017/978-4-7741-9205-5
https://gihyo.jp/dp/ebook/2017/978-4-7741-8845-4
https://gihyo.jp/dp/ebook/2017/978-4-7741-9201-7
https://gihyo.jp/dp/ebook/2017/978-4-7741-9203-1
https://gihyo.jp/dp/ebook/2017/978-4-7741-9196-9
https://gihyo.jp/dp/

In [24]:
%%writefile python_crawler_3.py

# 一覧ページからURLの一覧を抜き出す(3)
# あとで利用しやすいよう関数をつかってリファクタリングしておく

import requests
import lxml.html

def main():
    """
    クローラーのメインの処理
    """
    response = requests.get('https://gihyo.jp/dp')
    # scrape_list_page()関数を呼び出し、ジェネレーターイテレーターを取得
    urls = scrape_list_page(response)
    for url in urls: # ジェネレーターイテレーターはlistなどと同様に繰り返し可能
        print(url)

def scrape_list_page(response):
    """
    一覧ページのResponseから詳細ページのURLを抜き出すジェネレーター関数
    """
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url) # すべてのリンクを絶対URLに変換する

    # id="listBook"である要素の子孫のa要素のみを取得する
    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url # yield文でジェネレーターイテレーターの要素を返す
        
if __name__ == '__main__':
    main()

Writing python_crawler_3.py


In [ ]:
!python python_crawler_3.py

In [ ]:
# 詳細ページからスクレイピングする(クロール前のテスト)

In [3]:
%%writefile python_crawler_4.py

# 詳細ページからスクレイピングする(1)

import requests
import lxml.html


def main():
    session = requests.Session() # 複数のページをクロールするのでSessionを使う
    response = session.get('https://gihyo.jp/dp')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url) # Sessionを使って詳細ページを取得
        ebook = scrape_detail_page(response) # 詳細ページからスクレイピングして電子書籍の情報を得る
        print(ebook) # 電子書籍の情報を表示
        break # まず1ページだけで試すためbreak文でループを抜ける
        
        
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    
    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url
        
        
def scrape_detail_page(response):
    """
    詳細ページのResponseから電子書籍の情報をdictで取得する
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url, # URL
        'title': root.cssselect('#bookTitle')[0].text_content(), # タイトル
        'price': root.cssselect('.buy')[0].text, # 価格(.textで直接の子である文字列のみを取得)
        'content': [h3.text_content() for h3 in root.cssselect('#content > h3')], # 目次
    }
    return ebook # dictを返す


if __name__ == '__main__':
    main()

Overwriting python_crawler_4.py


In [4]:
!python python_crawler_4.py

{'url': 'https://gihyo.jp/dp/ebook/2017/978-4-7741-9243-7', 'title': 'スピードマスター1時間でわかる エクセル データ分析 超入門', 'price': '1,000円 ', 'content': ['1章\u3000データ分析をスムーズに進めてビジネスをレベルアップさせるには', '2章\u3000集団の全体像や特徴を見きわめ，代表値や全体の中での位置を求める', '3章\u3000複数の値どうしの関係を調べ，将来の値を予測する', '4章\u3000営業活動や販売促進，トラブル対策の戦略を立てる']}


In [8]:
%%writefile python_crawler_5.py

# 詳細ページからスクレイピングする(2)
#  不要な空白や改行は削除したい

import re
import requests
import lxml.html


def main():
    session = requests.Session() # 複数のページをクロールするのでSessionを使う
    response = session.get('https://gihyo.jp/dp')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url) # Sessionを使って詳細ページを取得
        ebook = scrape_detail_page(response) # 詳細ページからスクレイピングして電子書籍の情報を得る
        print(ebook) # 電子書籍の情報を表示
        break # まず1ページだけで試すためbreak文でループを抜ける
        
        
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    
    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url
        
        
def scrape_detail_page(response):
    """
    詳細ページのResponseから電子書籍の情報をdictで取得する
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url, # URL
        'title': root.cssselect('#bookTitle')[0].text_content(), # タイトル
        'price': root.cssselect('.buy')[0].text.strip(), # 価格(.textで直接の子である文字列のみを取得、strip()で前後の空白を削除)
        'content': [normalize_spaces(h3.text_content()) for h3 in root.cssselect('#content > h3')], # 目次
    }
    return ebook # dictを返す


def normalize_spaces(s):
    """
    連続する空白を1つのスペースに置き換え、前後の空白は削除した新しい文字列を取得する
    """
    return re.sub(r'\s+', ' ', s).strip()


if __name__ == '__main__':
    main()

Overwriting python_crawler_5.py


In [6]:
!python python_crawler_5.py

{'url': 'https://gihyo.jp/dp/ebook/2017/978-4-7741-9243-7', 'title': 'スピードマスター1時間でわかる エクセル データ分析 超入門', 'price': '1,000円', 'content': ['1章 データ分析をスムーズに進めてビジネスをレベルアップさせるには', '2章 集団の全体像や特徴を見きわめ，代表値や全体の中での位置を求める', '3章 複数の値どうしの関係を調べ，将来の値を予測する', '4章 営業活動や販売促進，トラブル対策の戦略を立てる']}


In [ ]:
# 詳細ページをクロールする

In [10]:
%%writefile python_crawler_6.py

# 詳細ページをクロールする
#  不要な空白や改行は削除したい
# 1秒ごとに電子書籍の情報を取得して表示する

import time
import re
import requests
import lxml.html


def main():
    session = requests.Session() # 複数のページをクロールするのでSessionを使う
    response = session.get('https://gihyo.jp/dp')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1) # 1秒のウェイトを入れる
        response = session.get(url) # Sessionを使って詳細ページを取得
        ebook = scrape_detail_page(response) # 詳細ページからスクレイピングして電子書籍の情報を得る
        print(ebook) # 電子書籍の情報を表示
        
        
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    
    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url
        
        
def scrape_detail_page(response):
    """
    詳細ページのResponseから電子書籍の情報をdictで取得する
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url, # URL
        'title': root.cssselect('#bookTitle')[0].text_content(), # タイトル
        'price': root.cssselect('.buy')[0].text.strip(), # 価格(.textで直接の子である文字列のみを取得、strip()で前後の空白を削除)
        'content': [normalize_spaces(h3.text_content()) for h3 in root.cssselect('#content > h3')], # 目次
    }
    return ebook # dictを返す


def normalize_spaces(s):
    """
    連続する空白を1つのスペースに置き換え、前後の空白は削除した新しい文字列を取得する
    """
    return re.sub(r'\s+', ' ', s).strip()


if __name__ == '__main__':
    main()

Overwriting python_crawler_6.py


In [ ]:
!python python_crawler_6.py

In [ ]:
# スクレイピングしたデータを保存する

In [12]:
%%writefile python_crawler_final.py

# 詳細ページをクロールする
#  不要な空白や改行は削除したい
# 1秒ごとに電子書籍の情報を取得して表示する
# 取得したデータをMongoDBに保存する(あらかじめMongoDBを起動しておく)
# mongod --config /usr/local/etc/mongod.conf
# 2回目以降はクロール済みのURLはクロールしないようにする

import time
import re
import requests
import lxml.html
from pymongo import MongoClient


def main():
    """
    クローラーのメインの処理
    """
    
    client = MongoClient('localhost', 27017) # ローカルホストのMongoDBに接続する
    collection = client.scraping.ebooks # scrapingデータベースのebooksコレクションを得る
    # データを一意に識別するキーを格納するkeyフィールドにユニークなインデックスを作成する
    collection.create_index('key', unique=True)
    
    response = requests.get('https://gihyo.jp/dp') # 一覧ページを取得する
    urls = scrape_list_page(response) # 詳細ページのURL一覧を得る
    for url in urls:
        key = extract_key(url) # URLからキーを取得する
        
        ebook = collection.find_one({'key': key}) # MongoDBからkeyに該当するデータを探す
        if not ebook: # MongoDBに存在しない場合だけ、詳細ページをクロールする
            time.sleep(1) # 1秒のウェイトを入れる
            response = requests.get(url) # 詳細ページを取得
            ebook = scrape_detail_page(response) # 詳細ページからスクレイピングして電子書籍の情報を得る
            collection.insert_one(ebook) # 電子書籍の情報をMongoDBに保存する
            
        print(ebook) # 電子書籍の情報を表示
        
        
def scrape_list_page(response):
    """
    一覧ページのResponseから詳細ページのURLを抜き出す
    """
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    
    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url
        
        
def scrape_detail_page(response):
    """
    詳細ページのResponseから電子書籍の情報をdictで得る
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url, # URL
        'key': extract_key(response.url), # URLから抜き出したキー
        'title': root.cssselect('#bookTitle')[0].text_content(), # タイトル
        'price': root.cssselect('.buy')[0].text.strip(), # 価格(.textで直接の子である文字列のみを取得、strip()で前後の空白を削除)
        'content': [normalize_spaces(h3.text_content()) for h3 in root.cssselect('#content > h3')], # 目次
    }
    return ebook # dictを返す


def extract_key(url):
    """
    URLからキー(URLの末尾のISBN)を抜き出す
    """
    m = re.search(r'/([^/]+)$', url)
    return m.group(1)


def normalize_spaces(s):
    """
    連続する空白を1つのスペースに置き換え、前後の空白は削除した新しい文字列を取得する
    """
    return re.sub(r'\s+', ' ', s).strip()


if __name__ == '__main__':
    main()

Writing python_crawler_final.py


In [ ]:
!python python_crawler_final.py